In [2]:
## IMPORTS
import seaborn as sns
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Display the first few rows to get a feel for the data
job_tweets_raw_df = pd.read_csv("../data/Job_Tweets.csv")
job_tweets_raw_df.head(10)

,ID,Timestamp,User,Text,Hashtag,Retweets,Likes,Replies,Source,Location,Verified_Account,Followers,Following
0,1211798864656769025,2019-12-30 23:58:58+00:00,SipapuNM,Looking for an exciting job where you can ski ...,NaN,0,1,0,"<a href=""http://instagram.com"" rel=""nofollow"">...","20 miles SE of Taos, NM",False,3342,258
1,1211797371853705220,2019-12-30 23:53:02+00:00,LorettaOD1,Hail in Phoenix - no way! New opportunity at ...,['job'],0,0,0,"<a href=""http://jobvite.com"" rel=""nofollow"">Jo...",Arizona,False,63,129
2,1211795775363145728,2019-12-30 23:46:41+00:00,guajardo_celina,Rackspace is hiring! We are looking for Nation...,"['becomearacker', 'Rackspace', 'recruiting', '...",0,0,0,"<a href=""http://jobvite.com"" rel=""nofollow"">Jo...","Texas, USA",False,19,50
3,1211793355060981767,2019-12-30 23:37:04+00:00,SteveEckert_OTD,We are #hiring Administrative Assistant in Nan...,"['hiring', 'jobs', 'Nanuet']",0,0,0,"<a href=""http://www.ziprecruiter.com"" rel=""nof...","Orange County, California",False,966,1569
4,1211792689022349315,2019-12-30 23:34:25+00:00,HireLive,We are #hiring Family Service Counselor in Irv...,"['hiring', 'jobs', 'Irvine']",0,0,0,"<a href=""http://www.ziprecruiter.com"" rel=""nof...",United States,False,983,1251
5,1211792528388841473,2019-12-30 23:33:47+00:00,HireLive,We are #hiring Inside Sales Representative in ...,"['hiring', 'jobs', 'Irvine']",0,0,0,"<a href=""http://www.ziprecruiter.com"" rel=""nof...",United States,False,983,1251
6,1211791693005099008,2019-12-30 23:30:28+00:00,HireLive,We are #hiring Community Service Counselor in ...,"['hiring', 'jobs', 'Irvine']",0,0,0,"<a href=""http://www.ziprecruiter.com"" rel=""nof...",United States,False,983,1251
7,1211790931772477440,2019-12-30 23:27:26+00:00,HireLive,We are #hiring Outside Sales Representative in...,"['hiring', 'jobs', 'Irvine']",0,0,0,"<a href=""http://www.ziprecruiter.com"" rel=""nof...",United States,False,983,1251
8,1211789520473079809,2019-12-30 23:21:50+00:00,PRGWest,We are #hiring Workday HRIS Manager 19-00454 ...,"['hiring', 'jobs', 'Irvine']",0,0,0,"<a href=""http://www.ziprecruiter.com"" rel=""nof...",Southern California,False,829,1015
9,1211788232620412929,2019-12-30 23:16:43+00:00,GhLisa,We are #hiring Restaurant Manager - Hourly Man...,"['hiring', 'jobs', 'Laurel']",0,0,0,"<a href=""http://www.ziprecruiter.com"" rel=""nof...","Mississippi, USA",False,89,80


In [4]:
# Filter the dataset for the selected countries
ireland_df = job_tweets_raw_df.dropna()[job_tweets_raw_df.dropna()['Location'].str.contains("Ireland")]
germany_df = job_tweets_raw_df.dropna()[job_tweets_raw_df.dropna()['Location'].str.contains("Germany")]
france_df = job_tweets_raw_df.dropna()[job_tweets_raw_df.dropna()['Location'].str.contains("France")]
spain_df = job_tweets_raw_df.dropna()[job_tweets_raw_df.dropna()['Location'].str.contains("Spain")]


In [5]:
ireland_df = ireland_df.set_index("Timestamp")
germany_df = germany_df.set_index("Timestamp")
france_df = france_df.set_index("Timestamp")
spain_df = spain_df.set_index("Timestamp")

<h1> Supervised Learning: </h1>

In [6]:
# Define 'high' engagement as a tweet with likes greater than the 75th percentile
high_engagement_threshold = job_tweets_raw_df['Likes'].quantile(0.75)
job_tweets_raw_df['High_Engagement'] = (job_tweets_raw_df['Likes'] > high_engagement_threshold).astype(int)

# Now, you can use 'High_Engagement' as your target variable


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV

# Define 'High_Engagement' as a new target variable
high_engagement_threshold = job_tweets_raw_df['Likes'].quantile(0.75)
job_tweets_raw_df['High_Engagement'] = (job_tweets_raw_df['Likes'] > high_engagement_threshold).astype(int)

# Features and target variable
X = job_tweets_raw_df[['Followers', 'Following', 'Retweets']]  # example features
y = job_tweets_raw_df['High_Engagement']  # target variable

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize classifier and grid search
rf_classifier = RandomForestClassifier(random_state=42)
param_grid = {'n_estimators': [100, 200, 300], 'max_depth': [5, 10, 15], 'min_samples_split': [2, 5, 10]}
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Predict and evaluate
best_rf_model = grid_search.best_estimator_
y_pred = best_rf_model.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


0.8769
              precision    recall  f1-score   support

           0       0.89      0.97      0.92      7801
           1       0.83      0.56      0.67      2199

    accuracy                           0.88     10000
   macro avg       0.86      0.76      0.79     10000
weighted avg       0.87      0.88      0.87     10000



The output appears to be a classification report from a machine learning model, which includes various performance metrics. Here's how to interpret each part of the report:

Precision (0 class / 1 class):

For the '0' class (low engagement), the precision is 0.89, meaning that 89% of the tweets predicted as low engagement were actually low engagement.
For the '1' class (high engagement), the precision is 0.83, indicating that 83% of the tweets predicted as high engagement were actually high engagement.
Recall (0 class / 1 class):

The recall for the '0' class is 0.97, which indicates that the model successfully identified 97% of all actual low engagement tweets.
The recall for the '1' class is much lower at 0.56, suggesting that the model identified only 56% of all actual high engagement tweets.
F1-Score (0 class / 1 class):

The F1-score for the '0' class is 0.92, showing a strong balance between precision and recall for low engagement tweets.
The F1-score for the '1' class is 0.67, which is decent but indicates that the model is not as effective at predicting high engagement tweets as it is at predicting low engagement tweets.
Support (0 class / 1 class):

The support is the number of actual occurrences of each class in the dataset. There were 7801 low engagement tweets and 2199 high engagement tweets.
Accuracy:

The overall accuracy of the model is 0.88, meaning it correctly predicted the engagement level for 88% of the tweets in the test set.
Macro Avg:

The macro average calculates metrics independently for each class and then takes the average, treating all classes equally. The macro average for precision, recall, and F1-score are 0.86, 0.76, and 0.79 respectively, which suggests moderate overall performance.
Weighted Avg:

The weighted average takes into account the support of each class. This means that the performance on the more frequent class (low engagement) influences the metric more. The weighted averages for precision, recall, and F1-score are all 0.87.
Comments:

The model performs well in classifying low engagement tweets, as indicated by high precision, recall, and F1-score for the '0' class.
However, it struggles with classifying high engagement tweets, especially in terms of recall, which is only 0.56. This means it misses a significant portion of the high engagement tweets (44%).
The imbalance in the dataset (7801 low engagement vs. 2199 high engagement) may contribute to the model's bias towards predicting low engagement more accurately.
To improve model performance, especially for the '1' class, you may consider:
Collecting more balanced data.
Using oversampling techniques for the minority class or undersampling for the majority class.
Adjusting the decision threshold to increase recall for the '1' class.
Exploring different feature sets or more complex models that might capture the nuances of high engagement tweets better.
Using cost-sensitive learning or modifying class weights to penalize the misclassification of the minority class more heavily.
The high overall accuracy shows promise, but the recall for the high engagement tweets needs attention to make the model more reliable for practical use.

<h1> Sentiment Analysis: </h1>

In [7]:
from textblob import TextBlob

# Define a function to calculate sentiment
def get_tweet_sentiment(text):
    # Create a TextBlob object
    analysis = TextBlob(text)
    # Set sentiment
    if analysis.sentiment.polarity > 0:
        return 1  # positive sentiment
    else:
        return 0  # neutral or negative sentiment

# Apply the function to each tweet
job_tweets_raw_df['Sentiment'] = job_tweets_raw_df['Text'].apply(get_tweet_sentiment)

# Now, you can use 'Sentiment' as your target variable for classification


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Define sentiment for each tweet
job_tweets_raw_df['Sentiment'] = job_tweets_raw_df['Text'].apply(get_tweet_sentiment)

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X = tfidf_vectorizer.fit_transform(job_tweets_raw_df['Text'])
y = job_tweets_raw_df['Sentiment']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

# Predict and evaluate
y_pred = nb_classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


0.8474
              precision    recall  f1-score   support

           0       0.90      0.83      0.87      5908
           1       0.78      0.87      0.82      4092

    accuracy                           0.85     10000
   macro avg       0.84      0.85      0.84     10000
weighted avg       0.85      0.85      0.85     10000



The output is a classification report for a sentiment analysis model. Here's an interpretation of the results:

Precision (0 class / 1 class):

The model has a precision of 0.90 for the '0' class, indicating that when it predicts a tweet has a neutral/negative sentiment, it is correct 90% of the time.
For the '1' class (positive sentiment), the precision is 0.78, meaning that 78% of tweets predicted as positive are correctly classified.
Recall (0 class / 1 class):

The recall for the '0' class is 0.83, which means that the model identifies 83% of all actual neutral/negative sentiment tweets correctly.
For the '1' class, the recall is higher at 0.87, suggesting that the model is able to identify 87% of all actual positive sentiment tweets.
F1-Score (0 class / 1 class):

The F1-score, which is a balance between precision and recall, is 0.87 for the '0' class and 0.82 for the '1' class. Both scores are relatively high, indicating a good balance between precision and recall for both sentiment classes.
Support (0 class / 1 class):

The support numbers indicate a relatively balanced dataset, with 5998 instances in the '0' class and 4092 instances in the '1' class.
Accuracy:

The overall accuracy of the model is 0.85, meaning it correctly classified the sentiment of the tweets 85% of the time across both classes.
Macro Avg:

The macro average scores are 0.84 for precision, 0.85 for recall, and 0.84 for F1-score. These values suggest that the model performs comparably on both classes without significant bias towards one class.
Weighted Avg:

The weighted average takes into account the support of each class, providing a performance metric that reflects the class distribution. With scores of 0.85 across precision, recall, and F1-score, the model shows solid performance weighted by the class distribution.
Comments:

The sentiment analysis model demonstrates strong performance in both identifying positive sentiment and distinguishing neutral/negative sentiment in tweets.
The balanced nature of the dataset is reflected in the macro and weighted averages being very close, indicating that the model's performance is not biased by class imbalance.
The F1-scores are high for both classes, which is desirable in a sentiment analysis context where both false positives and false negatives can be equally concerning.
One area to consider is the slightly lower precision for positive sentiments (0.78) compared to the precision for neutral/negative sentiments (0.90), which suggests that there might be some room for improvement in correctly identifying positive sentiments.
Given the high recall for positive sentiments (0.87), the model is robust in capturing most of the positive content, which is often a priority in sentiment analysis to understand customer satisfaction or positive feedback.
In conclusion, the sentiment analysis model performs well in classifying sentiments from the tweets with good accuracy and balance between classes. These results can provide confidence in using the model for analyzing public sentiment on social media regarding public sector employment and earnings.

<h1> General Notes: </h1>

The provided Python code snippets form a crucial part of our data analytics pipeline, aimed at extracting actionable insights from the tweet dataset related to Public Sector Employment and Earnings. The first snippet focuses on identifying tweets with high user engagement, a key indicator of the content's reach and impact. The second snippet delves into sentiment analysis, categorizing the general opinion expressed in the tweets as positive or neutral/negative. These analyses serve to inform strategies for social media engagement and public sentiment monitoring.

1. Supervised Learning for Engagement Prediction:

Purpose:
The goal here is to predict which public sector job tweets are likely to receive high engagement, defined by the number of 'Likes'. By identifying characteristics of highly engaged tweets, we can provide recommendations to increase the visibility and impact of future communications.

Code Description:

The code begins by determining a threshold for 'high engagement' based on the distribution of 'Likes'. We use the 75th percentile as a benchmark, meaning that tweets with 'Likes' above this value are considered highly engaged.
A new binary target variable, 'High_Engagement', is created where tweets above this threshold are labeled 1 (high engagement) and all others are 0 (low engagement).
We prepare the dataset for machine learning by selecting relevant features that might predict engagement, such as 'Followers', 'Following', and 'Retweets'.
The dataset is split into a training set for model building and a testing set for evaluation.
We utilize a RandomForestClassifier for its robustness and ability to handle non-linear relationships. GridSearchCV is employed to tune hyperparameters and find the best model configuration.
The best model from the grid search is then used to make predictions on the test set.
Finally, we evaluate the model's performance using accuracy score and a classification report, which includes precision, recall, and F1-score for both classes (high and low engagement).


2. Sentiment Analysis:

Purpose:
The sentiment analysis model aims to classify the sentiment of each tweet as either positive or neutral/negative. This information can be vital for understanding public perception of the public sector employment landscape.

Code Description:

The TextBlob library is utilized to calculate the sentiment polarity of each tweet, which is a measure of the sentiment's strength and direction (positive or negative).
We define positive sentiment as having a polarity score greater than zero and assign a corresponding label of 1, while all other tweets are labeled as 0.
The text data is transformed into a numerical format using TfidfVectorizer, which reflects the importance of words in the context of the entire dataset.
We use the MultinomialNB (Naive Bayes) classifier, which is particularly suited for text classification tasks due to its basis in probability.
After training the model on the training set, we make predictions on the test set and evaluate the model using the accuracy score and a classification report.
Conclusion:

These machine learning models are instrumental in uncovering trends and patterns within the tweet dataset. By predicting engagement and analyzing sentiment, we gain valuable insights into how public sector employment is viewed and discussed on social media platforms. The findings from these models can be leveraged to optimize communication strategies, enhance public relations efforts, and monitor the effectiveness of employment campaigns. Challenges encountered, model limitations, and recommendations for further analysis are detailed in the attached appendices.